In [1]:
import pandas as pd
import numpy
import matplotlib.pyplot as plt
import seaborn as sns
import dill

from python.mappings import FOOTBALL_LEAGUES_MAPPING, TEAMS_RANKING_MAPPING, EUROPEAN_TOURNAMENTS_MAPPING, PLAYERS_OUTSIDE_TOP5

from scipy.stats import norm, skew
from scipy.stats import probplot
from scipy.stats import f_oneway, shapiro, levene, kruskal

from scipy.special import boxcox1p
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.impute import SimpleImputer

from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split, ShuffleSplit, cross_validate, KFold, cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint as sp_randint

from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge, Lasso, ElasticNet, BayesianRidge
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR, NuSVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone

from sklearn.metrics import root_mean_squared_error, root_mean_squared_log_error, r2_score, mean_absolute_percentage_error
from sklearn.metrics import make_scorer

%matplotlib inline

In [2]:
# Constant variables
TABLE_SIZE = 1546
TRAIN_RATIO = 0.8

In [3]:
# Joins train and test datasets together to simplify the process
def concat_df(df1, df2):
    return pd.concat([df1, df2], ignore_index=True)

# Divides the overall dataset into train and test 
def divide_df(df):
    train_size = int(TABLE_SIZE*TRAIN_RATIO)
    
    return df.loc[:train_size - 1], df.loc[train_size:] 

In [4]:
train_data = pd.read_csv('data/train_stats.csv')
test_data = pd.read_csv('data/test_stats.csv')
all_data = concat_df(train_data, test_data)
df_train, df_test = divide_df(all_data)

In [5]:
clean_sheets = pd.read_csv('data/footballers_clean_sheets.csv')
popularity = pd.read_csv('data/footballers_popularity.csv')

In [6]:
all_data = all_data.merge(right=clean_sheets, on=['Name'], how='left')
all_data = all_data.merge(right=popularity, on=['Name'], how='left')

In [7]:
all_data['Clean Sheets %'].fillna(value=0, inplace=True)
all_data['Popularity'].fillna(value=1, inplace=True)

**1. PREPARATION STEP: Cleaning the data**

In [8]:
all_data.drop(['Player_URL', 'Team_URL'], axis=1, inplace=True)

In [9]:
# Cleans the columns containing commas 
def clean_comma(column):
    column = column.strip()
    column = column[:column.find(',')] + '' + column[column.find(',') + 1:]
    return column

# Cleans the columns containing '\t' symbol
def clean_tab(column):
    return column.strip().replace('\t', '')

In [10]:
comma_columns = ['Team', 'Position']
tab_columns = ['Goals', 'Assists', 'Yel', 'Red', 'SpG', 'AerialsWon', 'MotM', 'Tackles',
               'Inter', 'Fouls', 'Offsides', 'Clear', 'Drb_x', 'Blocks', 'OwnG', 'KeyP',
               'Drb_y', 'Fouled', 'Off', 'Disp', 'UnsTch', 'Crosses', 'LongB', 'ThrB']

for column in comma_columns:
    all_data[column] = all_data[column].apply(clean_comma)
    
for column in tab_columns:
    all_data[column] = all_data[column].apply(clean_tab)

In [11]:
# Some columns with integer values contain symbol '-' instead of 0. The function fixes it
def remove_dashes(column):
    if column == '-':
        return 0
    return column

In [12]:
dash_columns = ['Goals', 'Assists', 'Yel', 'Red', 'SpG', 'AerialsWon', 'MotM', 'Tackles',
                'Inter', 'Fouls', 'Offsides', 'Clear', 'Drb_x', 'Drb_y', 'Blocks', 'OwnG', 'KeyP',
                'Fouled', 'Off', 'Disp', 'UnsTch', 'Crosses', 'LongB', 'ThrB']

for column in dash_columns:
    all_data[column] = all_data[column].apply(remove_dashes)

In [13]:
# Now we have 'Value' variable containing market values in thousands or millions, but we want to get just an integer
def value_scaling(value):
    value = value.strip()
    
    if value.endswith('k'):
        return int(float(value[value.find('€')+1:value.find('k')]) * 10**3)
    elif value.endswith('m'):
        return int(float(value[value.find('€')+1:value.find('m')]) * 10**6)

In [14]:
all_data['Value'] = all_data['Value'].apply(value_scaling)

In [15]:
# We can divide 'Apps' variable because the number of appearances in starting squad is demonstrated in parentheses 
def appearances_division(apps):
    return apps[:apps.find('(')], apps[apps.find('(')+1:apps.find(')')]

In [16]:
all_data['Overall_Apps'], all_data['Start_Apps'] = zip(*all_data['Apps'].apply(appearances_division))

In [17]:
all_data.drop(['Apps'], axis=1, inplace=True)

In [18]:
# It would be better if we replace 'Forward', 'Midfielder' with their short forms (FW, M)

#all_data['Position'].value_counts()

def position_mapping(position):
    position = position.strip()
    
    if position == 'Forward': 
        return 'FW'
    elif position == 'Midfielder': 
        return 'M(C)'
    return position

all_data['Position'] = all_data['Position'].apply(position_mapping)

In [19]:
# We can divide 'Position' column into two positions (if some player has only one position, he`ll get NaN value for the second one).#
def position_division(position):
    
    # only two main positions, that`s enough
    if position.count(',') >= 2:
        while position.count(',') != 1:
            position = position[:position.rfind(',')]
          
    if position.find(',') != -1:
        return pd.Series([position[:position.find(',')].strip(), position[position.find(',')+1:]]).values
    return pd.Series([position.strip(), numpy.nan]).values

# Center (C), Right (R) or Left (L)? This information is contained in parentheses
def position_side(position):
    
    # numpy.nan has 'float' type
    if type(position) != float:
        if position.find('(') != -1:
            sides = tuple(position[position.find('(')+1:position.find(')')])
            position = position[:position.find('(')].strip(),
            
            return position + sides + tuple([numpy.nan] * (3 - len(sides)))
        else:
            position = position.strip(),

            return position + tuple([numpy.nan] * 3)
    else:
        return tuple([numpy.nan] * 4)

In [20]:
all_data['Position_1'], all_data['Position_2'] = zip(*all_data['Position'].apply(position_division))

In [21]:
all_data['Position_1'], all_data['Side_11'], all_data['Side_12'], all_data['Side_13'] = zip(*all_data['Position_1'].apply(position_side))
all_data['Position_2'], all_data['Side_21'], all_data['Side_22'], all_data['Side_23'] = zip(*all_data['Position_2'].apply(position_side))

In [22]:
all_data.drop(['Position'], axis=1, inplace=True)

In [24]:
# Time to change column types
float_type = ['SpG', 'AerialsWon', 'Tackles', 'Inter', 'Fouls', 'Offsides',
              'Clear', 'Drb_x', 'Blocks', 'KeyP', 'Drb_y', 'Fouled', 'Off',
              'Disp', 'UnsTch', 'Crosses', 'LongB', 'ThrB']
int_type = ['Goals', 'Assists', 'Yel', 'Red', 'MotM', 'OwnG', 'Overall_Apps', 'Start_Apps']

all_data[float_type] = all_data[float_type].astype('float')
all_data[int_type] = all_data[int_type].astype('int64')

In [25]:
# Insert 'Value' variable at the end of the dataset
value_column = all_data.pop('Value')
all_data['Value'] = value_column

# Insert 'Position_2' variable before the variable 'Side_21'
pos2_column = all_data.pop('Position_2')
index = all_data.columns.get_loc('Side_21')
all_data.insert(index, 'Position_2', pos2_column)

In [26]:
all_data = all_data[all_data['Value'] < 100000000]
all_data = all_data[(all_data['Age'] < 32) | (all_data['Popularity'] < 3)]

In [27]:
all_data[all_data['Name'] == 'Jacob Murphy']

,Name,Team,Age,Mins,Goals,Assists,Yel,Red,SpG,PS,...,Start_Apps,Position_1,Side_11,Side_12,Side_13,Position_2,Side_21,Side_22,Side_23,Value
1275,Jacob Murphy,Newcastle,29,1194,3,7,1,0,1.4,73.6,...,7,D,R,NaN,NaN,M,L,R,NaN,15000000


**2. EXPLORATORY DATA ANALYSIS AND FEATURE ENGINEERING**

**GLOSSARY. DESCRIPTION OF THE VARIABLES**

- SpG - Shots per game
- PS(%) - Percentage of successful passes
- AerialsWon, Aerial - Header in a direct contest with an opponent
- MotM - Man of the Match
- Tackle - Dispossessing an opponent, whether the tackling player comes away with the ball or not
- Interception (Inter) - Preventing an opponent's pass from reaching their teammates
- Fouls - How often a player commits a foul
  Fouled - How often a player gets fouled
- Offsides - It`s referred to "offside won" - the last man to step up to catch an opponent in an offside position
  Off - How often a player gets in offside position
- Clearance (Clear) - Action by a defending player that temporarily removes the attacking threat on their goal/that effectively alleviate pressure on their goal
- Drb_x - How often a player gets dribbled (being dribbled past by an opponent without winning a tackle)
  Drb_y - Frequency of successful dribbles
- Blocks - The number of blocked shots per game
- OwnG - Own goal
- KeyP - Key passes (the final pass leading to a shot at goal from a teammate)
- Dispossessed (Disp) - How often a player gets tackled by an opponent without attempting to dribble past them
- UnsTch - ???
- Average Passes (AvgP) - Average number of passes attempted (short passes, long balls, through balls, crosses)
- Cross - An attempted/accurate pass from a wide position to a central attacking area
- Long Ball (LongB) - An attempted/accurate pass of 25 yards or more
- Through Ball (ThrB) - An attempted/accurate pass between opposition players in their defensive line to find an onrushing teammate (running through on goal)
- Apps - appearances on the football field

In [ ]:
df_train, df_test = divide_df(all_data)

Let`s start with categorical variables.

We see that there are a lot of missing values for 'Side_2X' and 'Side_13' variables (>80%). It describes where a player is located on a football pinch (left, right or center). But it can be useful to create a variable that shows how much place some player "occupies", on how many sides he is able to play. If it isn`t shown (all three variables are NaN values), then we will set 1.

In [28]:
def position_sides(s1, s2, s3):
    sides_list = [s1, s2, s3]
    
    while numpy.nan in sides_list:
        sides_list.remove(numpy.nan)
    
    if len(sides_list) == 0:
        return 1
    return len(sides_list)

In [29]:
all_data['Position_1_Sides'] = all_data.apply(lambda x: position_sides(x['Side_11'], x['Side_12'], x['Side_13']), axis=1)
all_data['Position_2_Sides'] = all_data.apply(lambda x: position_sides(x['Side_21'], x['Side_22'], x['Side_23']), axis=1)

Other variables have no missing values so we can continue in our research

'Team' variable can be very useful: we can extract a league in which a football player competes, and it is somehow influences his market value. For instance, we expect that in Premier League players cost more because of higher level of this championship and some other aspects. For this task we will use our mapping.

Also, we can divide our teams into three groups (A, B, C) based on their level ranking. Perhaps, players from "big teams" are more expensive 

In [30]:
all_data['League'] = all_data['Team'].map(FOOTBALL_LEAGUES_MAPPING)
all_data['Team_rank'] = all_data['Team'].map(TEAMS_RANKING_MAPPING)
all_data['European tournament'] = all_data['Team'].map(EUROPEAN_TOURNAMENTS_MAPPING)
all_data['Out of top5'] = all_data['Name'].isin(PLAYERS_OUTSIDE_TOP5)

Let`s check our presumptions by creating some plots!

In [31]:
order_rank = ['A-tier', 'B-tier', 'C-tier']

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 10), sharey=True)

sns.boxplot(all_data,
            x='League',
            y='Value',
            ax=ax[0])

sns.boxplot(all_data,
            x='Team_rank',
            y='Value',
            order=reversed(order_rank),
            ax=ax[1])

ax[0].set_title('The distribution of samples across the leagues')
ax[0].set_yticks(ticks=[0, 0.25e8, 0.5e8, 0.75e8, 1e8, 1.25e8, 1.5e8, 1.75e8], 
                 labels=['0m', '25m', '50m', '75m', '100m', '125m', '150m', '175m'])
ax[0].set_ylabel('Market value (in millions)')

ax[1].set_title('The distribution of samples across the team rankings')
ax[1].set_xlabel('Rank')
ax[1].set_ylabel('')

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))

sns.kdeplot(all_data,
            x='Value',
            hue='League')

ax.set_title('The distribution of market values across the leagues')
ax.set_xlabel('Market value (in millions)')
ax.set_xticks(ticks=[0, 0.5e8, 1e8, 1.5e8, 2e8], labels=['0', '50m', '100m', '150m', '200m'])

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))

sns.kdeplot(all_data,
            x='Value',
            hue='Team_rank',
            hue_order=order_rank)

ax.set_title('The distribution of market values based on the team rank')
ax.set_xlabel('Market value (in millions)')
ax.set_xticks(ticks=[0, 0.5e8, 1e8, 1.5e8, 2e8], labels=['0', '50m', '100m', '150m', '200m'])

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 14))

sns.swarmplot(all_data,
              x='League',
              y='Value',
              size=4,
              hue='Team_rank',
              hue_order=order_rank,
              ax=ax)

ax.set_title('The distribution of samples across the leagues and team rankings', fontsize=20)
ax.set_yticks(ticks=[0, 0.25e8, 0.5e8, 0.75e8, 1e8, 1.25e8, 1.5e8, 1.75e8], 
                 labels=['0m', '25m', '50m', '75m', '100m', '125m', '150m', '175m'])
ax.set_ylabel('Market value (in millions)', fontsize=18)
ax.xaxis.label.set_size(18)
ax.tick_params(axis='both', which='major', labelsize=15)
ax.legend(bbox_to_anchor=(1, 1), loc=2, fontsize=15)

plt.show()

On the first graph we can see that the distributions are almost the same across the leagues, but the Premier league has a little bit different one with higher market values than for other leagues.

From the second one we can notice that the distributions for A-ranked and B-ranked clubs are more heavy-tailed than for C-ranked teams, and in general, the majority of players in "small clubs" are located on the left part of the graph.

In [ ]:
# Creates a table with different descriptive statistics for different groups of a categorical variable 
def descriptive_table(table, variable):
    indices=table.groupby(variable)['Value'].mean().index
    counts = table.groupby(variable)['Value'].count().values
    means = table.groupby(variable)['Value'].mean().values
    stds = table.groupby(variable)['Value'].std().values
    medians = table.groupby(variable)['Value'].median().values
    quantile_ninty = table.groupby(variable)['Value'].quantile(q=0.9).values
    
    table =  pd.DataFrame({'Sample size': counts,
                           'Mean': means,
                           'St. deviation': stds,
                           'Median': medians,
                           '90% quantile': quantile_ninty
                           }, index=indices)
    return table    

In [ ]:
for item in ['League', 'Team_rank']:
    print(f"Variable: {item}")
    display(descriptive_table(all_data, item))

**Idea**: We can create a variable that indicates whether some footballer plays in Premier League instead of 'League' column

In [32]:
all_data['From EPL'] = all_data['League'] == 'Premier League'

In [33]:
all_data['Champion'] = all_data['Team'].isin(['Inter', 'Real Madrid', 'Leverkusen', 'Man City', 'PSG'])

Now we will work with positions. It can be very useful to divide player`s positions into four groups (attack, midfield, defence and goalkeeper) because we can expect that the players in attack line cost more than goalkeepers. We will only operate with the main position and create 'Position' column.

In [34]:
position_mapping = {'FW': 'Attack',
                    'AM': 'Midfield',
                    'M': 'Midfield',
                    'DMC': 'Midfield',
                    'D': 'Defence',
                    'GK': 'Goalkeeper'}

all_data['Position'] = all_data['Position_1'].map(position_mapping)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 10), sharey=True)

position_rank = ['Goalkeeper', 'Defence', 'Midfield', 'Attack']

sns.boxplot(all_data,
            x='Position',
            y='Value',
            order=position_rank,
            ax=ax[0])

sns.stripplot(all_data,
              x='Position',
              y='Value',
              order=position_rank,
              ax=ax[1])

fig.suptitle('The distribution of samples depending on positions', fontsize=20)
ax[0].set_yticks(ticks=[0, 0.25e8, 0.5e8, 0.75e8, 1e8, 1.25e8, 1.5e8, 1.75e8], 
                 labels=['0m', '25m', '50m', '75m', '100m', '125m', '150m', '175m'])
ax[0].set_ylabel('Market value (in millions)')

ax[1].set_ylabel('')

plt.show()

In [ ]:
display(descriptive_table(all_data, 'Position'))

In [ ]:
# Significance test for some categorical variable
def significance_test(variable, transformation=False, test_type=None):
    groups = []

    for group in all_data[variable].unique():
        sample = all_data.query(variable + '== @group')['Value'].values
        
        if transformation == 'log': 
            sample = numpy.log(sample + 1)
        elif transformation == 'sqrt': 
            sample = numpy.sqrt(numpy.abs(sample))
            
        print(f'Shapiro-Wilk test: {shapiro(sample)}')
        groups.append(sample)

    print(f'Levene: {levene(*groups)}')
    
    if test_type == 'anova':
        print(f'ANOVA test: {f_oneway(*groups)}')
    elif test_type == 'kruskal':
        for g in groups:
            numpy.sort(g)
            
        print(f'Kruskal-Wallis test: {kruskal(*groups)}')

In [ ]:
significance_test('Position')

The samples aren`t distributed normally and their variances differ significantly, but we can use transformations

In [ ]:
for tr in ['log', 'sqrt']:
    significance_test('Position', transformation=tr)
    print()
    print('=' * 25)
    print()

The samples` distribution is still abnormal, but their variances are homogenous, so we can try to use Kruskal-Wallis test (we cannot use ANOVA test)

In [ ]:
print('Without transformation: ')
significance_test('Position', transformation=False, test_type='kruskal')

In [ ]:
print('With log transformation: ')
significance_test('Position', transformation='log', test_type='kruskal')

Since Kruskal-Wallis test shows significant result, we can leave this variable

Let`s check 'Side_11' variable because it is footballer's main side (location where he usually plays)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

side_rank = ['L', 'C', 'R']

sns.boxplot(all_data,
            x='Side_11',
            y='Value',
            order=side_rank,
            ax=ax)

fig.suptitle('The distribution of samples based on the side on the football pinch', fontsize=20)
ax.set_yticks(ticks=[0, 0.25e8, 0.5e8, 0.75e8, 1e8, 1.25e8, 1.5e8, 1.75e8], 
                 labels=['0m', '25m', '50m', '75m', '100m', '125m', '150m', '175m'])
ax.set_ylabel('Market value (in millions)')
ax.set_xlabel('Side')

plt.show()

In [ ]:
descriptive_table(all_data, 'Side_11')

In [35]:
all_data['Side_11'].fillna(all_data['Side_11'].mode().values[0], inplace=True)

In [36]:
all_data.rename(columns={'Side_11': 'Main side'}, inplace=True)

This variable is not so useful as the previous ones. There are too little data for Left and Right sides, and we can expect that 'Side_12', 'Side_13', etc. are even less informative, so in the future we can drop them, leaving only 'Main_side' (it is the same as 'Side_11') variable.

Now we will work with 'Position_1_Sides' and 'Position_2_Sides' and create some interesting (at least we hope) plots

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))

sns.kdeplot(all_data,
            x='Value',
            hue='Position_1_Sides',
            palette=['blue', 'green', 'orange'])

ax.set_title('The distribution of market values depending on the number of positions a footballer occupies')
ax.set_xlabel('Market value (in millions)')
ax.set_xticks(ticks=[0, 0.5e8, 1e8, 1.5e8, 2e8], labels=['0', '50m', '100m', '150m', '200m'])

plt.show()

Now let`s work with numerical variables  

In [ ]:
all_data.describe()

In [ ]:
df_train, df_test = divide_df(all_data)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))

sns.histplot(df_train,
            x='Age',
            kde=True,
            ax=ax,
            label='train set')

sns.histplot(df_test,
            x='Age',
            kde=True,
            bins=18,
            ax=ax,
            label='test set')

ax.set_title('The distribution of players by age')
ax.set_xlabel('Age')
ax.legend()

plt.show()

In [37]:
all_data['Very old'] = all_data['Age'] >= 32

In [ ]:
df_train, df_test = divide_df(all_data)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))

sns.boxplot(df_train,
            x='Age',
            y='Value',
            ax=ax)

ax.set_title('The relation between age and value (train set)')
ax.set_xlabel('Age')
ax.set_yticks(ticks=[0, 0.25e8, 0.5e8, 0.75e8, 1e8, 1.25e8, 1.5e8, 1.75e8], 
                 labels=['0m', '25m', '50m', '75m', '100m', '125m', '150m', '175m'])
ax.set_ylabel('Market value (in millions)')

plt.show()

In [ ]:
descriptive_table(df_train, 'Age')

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))

sns.histplot(df_train,
            x='Mins',
            kde=True,
            ax=ax,
            label='train set')

sns.histplot(df_test,
            x='Mins',
            kde=True,
            bins=16,
            ax=ax,
            label='test set')

ax.set_title('The distribution of players by minutes played')
ax.set_xlabel('Minutes')
ax.legend()

plt.show()

In [ ]:
sns.jointplot(df_train,
              x='Mins',
              y='Value',
              kind="hex", 
              height=8,
              color="#FFF668")

plt.show()

In [ ]:
#all_data['Minutes for MotM'] = all_data['Mins'] / all_data['MotM']
#all_data['Minutes for Goal'] = all_data['Mins'] / all_data['Goals']
#all_data['Minutes for Assist'] = all_data['Mins'] / all_data['Assists']
#all_data['Minutes for KeyP'] = all_data['Mins'] / all_data['KeyP']

In [ ]:
df_train, df_test = divide_df(all_data)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))

sns.boxplot(df_train,
            x='Mins',
            y='Value',
            ax=ax)

ax.set_title('The distribution of market values depending on how many minutes a footballer played')
ax.set_xlabel('Mins')
ax.set_yticks(ticks=[0, 0.25e8, 0.5e8, 0.75e8, 1e8, 1.25e8, 1.5e8, 1.75e8], 
                 labels=['0m', '25m', '50m', '75m', '100m', '125m', '150m', '175m'])
ax.set_ylabel('Market value (in millions)')

plt.show()

In [ ]:
descriptive_table(df_train, 'Mins')

In [38]:
all_data['Goals'] = pd.cut(all_data['Goals'], bins=4)
all_data['Assists'] = pd.cut(all_data['Assists'], bins=4)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 10), sharey=True)

sns.boxplot(all_data,
            x='Goals',
            y='Value',
            ax=ax[0])

sns.boxplot(all_data,
            x='Assists',
            y='Value',
            ax=ax[1])

ax[0].set_title('The distribution of the market values (goals)')
ax[0].set_yticks(ticks=[0, 0.25e8, 0.5e8, 0.75e8, 1e8, 1.25e8, 1.5e8, 1.75e8], 
                 labels=['0m', '25m', '50m', '75m', '100m', '125m', '150m', '175m'])
ax[0].set_ylabel('Market value (in millions)')

ax[1].set_title('The distribution of the market values (assists)')
ax[1].set_ylabel('')

plt.show()

In [ ]:
for item in ['Goals', 'Assists']:
    print(f'Variable: {item}')
    display(descriptive_table(all_data, item))

Let`s drop redundant features or the features with missing values

In [39]:
all_data.drop(['Side_12', 'Side_13', 'Side_21', 'Side_22', 'League', 'Team',
               'Side_23', 'Position_1', 'Position_2', 'Position_2_Sides'], axis=1, inplace=True)

all_data, all_names = all_data.drop('Name', axis=1), all_data['Name']

train_names, test_names = divide_df(all_names) 

In [ ]:
all_data.sample(10)

In [ ]:
def pairplot_creator(table, index_start=0, index_end=40):
    new_table = table.iloc[:, index_start:index_end]
    new_table['Value'] = table['Value']
    
    sns.pairplot(new_table, height=2)
    plt.show()

In [ ]:
pairplot_creator(df_train, index_start=4, index_end=9)

In [41]:
# Insert 'Value' variable at the end of the dataset
value_column = all_data.pop('Value')
all_data['Value'] = value_column

In [42]:
df_train, df_test = divide_df(all_data)

In [ ]:
corr_table = df_train.corr(numeric_only=True)

fig, ax = plt.subplots(figsize=(20, 15))

sns.heatmap(corr_table,
            annot=True,
            fmt='.2f',
            ax=ax)

ax.set_title('Pairwise correlation plot', fontsize=20)
ax.tick_params(axis='both', which='major', labelsize=15)

plt.show()f

Let's check the variance of some variables and use filter methods of feature selection (in the final version of the notebook we didn`t use it :( )

In [43]:
low_variance_correlation = []

for item in all_data.columns:
    if (all_data[item].dtype == 'int64' or all_data[item].dtype == 'float') and not item.startswith('Minutes'):
        print(f'Variable: {item}')
        variance = all_data[item].values.var()
        correlation = numpy.corrcoef(all_data[item].values, all_data['Value'].values)[0][1]
        print(f'Its variance: {variance}')
        print(f'Its correlation with the target variable: {correlation}')
        print('=' * 25)
        
        if variance < 0.2 and correlation < 0.1:
            low_variance_correlation.append(item)

Variable: Age
Its variance: 16.22992888152291
Its correlation with the target variable: -0.32480813271727754
Variable: Mins
Its variance: 499480.52425897727
Its correlation with the target variable: 0.24684454725888857
Variable: Yel
Its variance: 7.229348766378711
Its correlation with the target variable: 0.09198648287442948
Variable: Red
Its variance: 0.17075891222418454
Its correlation with the target variable: -0.02522003841305955
Variable: SpG
Its variance: 0.427846627675709
Its correlation with the target variable: 0.351899663183083
Variable: PS
Its variance: 62.57501502218596
Its correlation with the target variable: 0.26199310700851086
Variable: AerialsWon
Its variance: 0.7295054551684621
Its correlation with the target variable: -0.02427373212302789
Variable: MotM
Its variance: 1.843871352032245
Its correlation with the target variable: 0.35050271825750795
Variable: Rating
Its variance: 0.06410839240639485
Its correlation with the target variable: 0.5768693761665808
Variable: T

In [ ]:
all_data.sample(5)

In [ ]:
all_data.info()

In [ ]:
all_data.columns

In [44]:
all_data = all_data[['Team_rank', 'Age', 'Popularity', 'Goals', 'Rating', 'From EPL', 'Mins', 'AvgP', 'PS', 'Assists', 'Value']]

**Feature transformation**

In [45]:
label_encoding = ['Goals', 'Assists', 'Team_rank']
num_features = list(set(all_data.columns) - set(label_encoding) - set(['Value']))

In [46]:
label_enc = LabelEncoder()

for label in label_encoding:
    all_data[label] = label_enc.fit_transform(all_data[label])

In [47]:
df_train, df_test = divide_df(all_data)

**3. LEARNING A MODEL**

This section will be divided into 3 parts. In the first part we will create the DecisionTree model to see how it makes decisions on the data and how well it performs. In the next part we will examine all the basic machine learning models in order to choose the best one. In the final part we will choose one of the algorithms and learn it via GridSearch with a larger hyperparameters grid 

In [48]:
df_train, df_test = divide_df(all_data)

X_train, y_train = df_train.drop('Value', axis=1), df_train['Value']
X_test, y_test = df_test.drop('Value', axis=1), df_test['Value']

feature_names = X_train.columns

n_folds = 5

In [ ]:
X_train

*1. Decision Tree*

In this part we will create a basic decision tree and then visualize it. This model is good at representativeness and interpretability so we must know how to deal with it

In [ ]:
# Train a basic model - without any hyperparameter fine-tuning
basic_decision_tree = DecisionTreeRegressor()

In [ ]:
basic_decision_tree.fit(X_train, y_train)

In [55]:
def regression_metrics(prediction, reality):
    maximum = max(reality)
    minimum = min(reality)
    
    print(f'RMSE: {root_mean_squared_error(prediction, reality)}')
    print(f'Normalized RMSE: {root_mean_squared_error(prediction, reality)/(maximum-minimum)}')
    print(f'log RMSE: {root_mean_squared_log_error(numpy.abs(prediction), reality)}')
    print(f'MAPE: {mean_absolute_percentage_error(prediction, reality)}')
    print(f'R2 score: {r2_score(prediction, reality)}')
    
def show_predictions(model):
    answers = pd.DataFrame([test_names.values, y_test.values, pd.Series(model.predict(X_test))]).T
    answers.rename(columns={0: 'Name', 1: 'Test value', 2: 'Predicted value'}, inplace=True)
    
    return answers

def show_importances(features, model):
    feature_importances = pd.DataFrame(model.feature_importances_, index=features, columns=['Importance'])
    feature_importances.sort_values(by='Importance', ascending=False, inplace=True)
    
    return feature_importances

def rmse_cv(model):
    kfolds = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train.values)
    rmse = numpy.sqrt(-cross_val_score(model, X_train.values, y_train, cv=kfolds, scoring='neg_mean_squared_error'))
    
    return rmse

In [ ]:
regression_metrics(basic_decision_tree.predict(X_train), y_train)

In [ ]:
regression_metrics(basic_decision_tree.predict(X_test), y_test)

In [ ]:
display(show_predictions(basic_decision_tree).sample(5))

We see that this model expectedly overfits the data. Let`s somehow adjust our algorithm

In [ ]:
vis_decision_tree = DecisionTreeRegressor(max_depth=3)

In [ ]:
vis_decision_tree.fit(X_train, y_train)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))

plot_tree(vis_decision_tree, feature_names=feature_names, filled=True, fontsize=8)

plt.show()

In [ ]:
display(show_importances(feature_names, vis_decision_tree).head(8))

The basic decision tree with max_depth = 3 makes its decisions based mainly on the rating, rank of the team in which a footballer plays and his age. So our new feature 'Team_rank' is very important for this ML algorithm. Let`s check the same thing for the first tree

In [ ]:
regression_metrics(vis_decision_tree.predict(X_train), y_train)

In [ ]:
regression_metrics(vis_decision_tree.predict(X_test), y_test)

This model has problem with underfitting the data. So let`s use the RandomizedSearch algorithm in order to get the best possible tree. It will be faster than usual GridSearch and may give better performance.

In [ ]:
decision_tree_param_grid = {
    'max_depth': [1, 3, 5, 7, 11, 13, 15],
    'min_samples_split': sp_randint(2, 16),
    'min_samples_leaf': sp_randint(2, 16)
}

In [ ]:
tune_decision_tree = RandomizedSearchCV(DecisionTreeRegressor(), decision_tree_param_grid, n_iter=300, cv=5, random_state=42)
tune_decision_tree.fit(X_train, y_train)

In [ ]:
best_decision_tree = tune_decision_tree.best_estimator_

In [ ]:
best_decision_tree.fit(X_train, y_train)

In [ ]:
show_importances(feature_names, best_decision_tree)

In [ ]:
regression_metrics(best_decision_tree.predict(X_train), y_train)

In [ ]:
regression_metrics(best_decision_tree.predict(X_test), y_test)

In [ ]:
display(show_predictions(best_decision_tree).sample(10))

This model performs better. But we see that it shows poor performance and doesn't use all the features we have. Let's check other ML algorithms and choose the best one

*2. Choosing a model*

In [ ]:
all_models = [
    DecisionTreeRegressor(),
    RandomForestRegressor(),
    BaggingRegressor(),
    AdaBoostRegressor(),
    GradientBoostingRegressor(),
    XGBRegressor(),
    ExtraTreesRegressor(),
    
    SVR(),
    NuSVR(),
    
    LinearRegression(),
    SGDRegressor(),
    Ridge(), 
    Lasso(), 
    ElasticNet(), 
    BayesianRidge()
]

In [ ]:
cv_splits = KFold(n_splits=5, shuffle=True)

scores_table = pd.DataFrame(columns=['train RMSE', 'test RMSE', 'train RMSE 3std', 'test RMSE 3std'])
predictions_table = pd.DataFrame(y_train, index=y_train.index)

for alg in all_models:
    alg_name = alg.__class__.__name__
    
    cv_results = cross_validate(alg, 
                                X_train, 
                                y_train,
                                cv=cv_splits,
                                scoring='neg_root_mean_squared_error',
                                return_train_score=True)
    
    scores_table.loc[alg_name, 'train RMSE'] = -cv_results['train_score'].mean()
    scores_table.loc[alg_name, 'test RMSE'] = -cv_results['test_score'].mean()
    #scores_table.loc[alg_name, 'train R2'] = cv_results['train_score'].mean()
    #scores_table.loc[alg_name, 'test R2'] = cv_results['test_score'].mean()
    
    scores_table.loc[alg_name, 'train RMSE 3std'] = cv_results['train_score'].std()*3
    scores_table.loc[alg_name, 'test RMSE 3std'] = cv_results['test_score'].std()*3
    #scores_table.loc[alg_name, 'train R2 3std'] = cv_results['train_score'].std()*3
    #scores_table.loc[alg_name, 'test R2 3std'] = cv_results['test_score'].std()*3
    
    alg.fit(X_train, y_train)
    
    predictions_table.loc[:, alg_name] = alg.predict(X_train)
    
scores_table.sort_values(by='test RMSE', ascending=True, inplace=True)

In [ ]:
display(scores_table)

We see that the best performance show ensemble and boosting methods like Gradient Boosting or Random Forest, but we also can notice that SVM algorithm obviously has an underfitting problem, so we can try to somehow fine-tune it (using kernels or adjusting the values of hyperparameters). But now we will try to use the RandomForestRegressor and find the best values of hyperparameters via GridSearchCV.

In [49]:
# GRID PARAMETERS

# 1. Lasso regression
lasso_grid_params = {
    'alpha': [0.001, 0.01, 0.1, 1, 3, 5, 7, 10, 15, 20, 25, 30, 40, 50, 60, 70, 80, 90, 100, 125, 150, 175,
              200, 250, 300, 400, 500, 600, 700, 800, 900, 1000, 1200, 1500, 1700, 2000, 2500, 3000,
              3500, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 12000, 14000, 16000, 18000, 20000,
              25000, 30000, 35000, 40000, 50000, 60000, 70000, 80000, 81500, 81510, 81520, 81530],
    'positive': [True, False]
}

# 2. Ridge regression
ridge_grid_params = {
    'alpha': [0.001, 0.01, 0.1, 1, 3, 5, 7, 10, 15, 17, 18, 18.2, 18.25, 18.26, 18.27, 18.28, 18.29, 18.3,
              18.31, 18.32, 18.33, 18.34, 18.35, 18.4, 18.5, 18.6, 19, 20, 22, 24, 25, 30, 40, 50, 60,
              70, 80, 90, 100, 125, 150, 175, 200, 250, 300, 400, 500, 600, 700, 800, 900, 1000, 1200,
              1500, 1700, 2000, 2500, 3000, 3500, 4000, 5000, 6000, 7000, 8000, 9000, 10000],
    'positive': [True, False]
}

# 3. ElasticNet
elastic_grid_params = {
    'alpha': [0.001, 0.01, 0.1, 1, 3, 5, 7, 10],
    'l1_ratio': numpy.arange(0.6, 0.9, 0.01)
}

# 4. KernelRidge regression
kernelridge_grid_params = {
    'kernel': ['linear', 'polynomial', 'rbf', 'sigmoid'],
    'gamma': [0.001, 0.01, 0.1, 1, 3, 5, 7, 10, 15, 20, 25, 30, 40, 50, 60, 70, 80, 90,
              100, 125, 150, 175, 200, 250, 300, 400, 500, 600, 700, 800, 900, 1000],
    'degree': [2, 3, 4, 5, 6, 7, 8, 9, 10],
    'coef0': [0.01, 0.1, 0.3, 0.5, 1, 1.5, 2, 2.5, 3, 4, 5, 7, 10]
}

# 5. GradientBoosting (n_iters = 3000)
gradient_grid_params = {
    'loss': ['squared_error', 'huber'],
    'n_estimators': [10, 30, 50, 70, 100, 300, 500, 1000],
    'learning_rate': numpy.arange(0, 1, 0.05),
    'max_depth': numpy.arange(1, 19, 2),
    'max_features': [None, 'sqrt'],
    'min_samples_split': numpy.arange(2, 21),
    'min_samples_leaf': numpy.arange(1, 19)
}

# 6. XGBoost
xgboost_grid_params = {
    'n_estimators': [10, 30, 50, 70, 100, 300, 500, 1000],
    'learning_rate': numpy.arange(0, 0.5, 0.025),
    'max_depth': numpy.arange(1, 19, 2),
    'gamma': [0.05, 0.07, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 1, 3, 5, 7, 10],
    'min_child_weight': [0.5, 1, 1.05, 1.15, 1.35, 1.5, 1.7, 2, 2.5, 3, 5, 7, 10, 15, 30, 50, 100, 200, 300, 500],
    'subsample': [0.5, 0.52, 0.54, 0.56, 0.58, 0.6, 0.63, 0.66, 0.69, 0.75, 0.8, 0.85, 0.9, 0.95, 1],
    'colsample_bytree':numpy.arange(0, 1, 0.05),
    'lambda': [0.001, 0.01, 0.1, 1, 3, 5, 7, 10, 15, 20, 25, 30, 40, 50, 60, 70, 80, 90,
              100, 125, 150, 175, 200, 250, 300, 400, 500, 600, 700, 800, 900, 1000],
    'alpha': [0.001, 0.01, 0.1, 1, 3, 5, 7, 10, 15, 20, 25, 30, 40, 50, 60, 70, 80, 90,
              100, 125, 150, 175, 200, 250, 300, 400, 500, 600, 700, 800, 900, 1000],
    'eval_metric': ['rmse']
}

# 7. LightGBM
lightgbm_grid_params = {
    'objective': ['regression', 'regression_l1', 'huber'],
    'n_estimators': [10, 30, 50, 70, 100, 300, 500, 1000],
    'learning_rate': numpy.arange(0, 0.5, 0.025),
    'max_depth': numpy.arange(1, 19, 2),
    'gamma': [0.05, 0.07, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 1, 3, 5, 7, 10],
    'min_samples_leaf': numpy.arange(1, 19),
    'min_child_weight': [0.5, 1, 1.05, 1.15, 1.35, 1.5, 1.7, 2, 2.5, 3, 5, 7, 10, 15, 30, 50, 100, 200, 300, 500],
    'subsample': [0.5, 0.52, 0.54, 0.56, 0.58, 0.6, 0.63, 0.66, 0.69, 0.75, 0.8, 0.85, 0.9, 0.95, 1],
    'subsample_freq': [0, 2, 3, 4, 5],
    'colsample_bytree': numpy.arange(0, 1, 0.05),
    'reg_lambda': [0.001, 0.01, 0.1, 1, 3, 5, 7, 10, 15, 20, 25, 30, 40, 50, 60, 70, 80, 90,
              100, 125, 150, 175, 200, 250, 300, 400, 500, 600, 700, 800, 900, 1000],
    'reg_alpha': [0.001, 0.01, 0.1, 1, 3, 5, 7, 10, 15, 20, 25, 30, 40, 50, 60, 70, 80, 90,
              100, 125, 150, 175, 200, 250, 300, 400, 500, 600, 700, 800, 900, 1000],
    'max_bin': [0.5, 1, 1.05, 1.15, 1.35, 1.5, 1.7, 2, 2.5, 3, 5, 7, 10, 15, 30, 50, 100, 200, 300, 500]
}

# 8. RandomForest
random_grid_params = {
    'max_depth': range(5, 14, 2),
    'min_samples_split': range(2, 11, 2),
    'min_samples_leaf': range(1, 11, 2),
    'max_features': numpy.arange(0.6, 1.1, 1),
    'n_estimators': [100]
}

In [50]:
# Lasso. Best params: alpha=81500, positive=False
lasso = make_pipeline(RobustScaler(), Lasso(alpha=81500,
                                            positive=False))

# Ridge. Best params: alpha=18.3, positive=False
ridge = make_pipeline(RobustScaler(), Ridge(alpha=18.3,
                                            positive=False))

# ElasticNet. Best params: alpha=0.1, l1_ratio=0.8100000000000002
elastic = make_pipeline(RobustScaler(), ElasticNet(alpha=0.1,
                                                   l1_ratio=0.8100000000000002,
                                                   positive=False))

# KernelRidge. Best params: kernel='polynomial', gamma=0.001, degree=3, coef0=31
kernelridge = KernelRidge(kernel='polynomial',
                          gamma=0.001,
                          degree=3,
                          coef0=31)

# GradientBoosting. Best params: loss='squared_error', n_estimators=600, learning_rate=0.05, min_samples_split=3, min_samples_leaf=3
gradient = GradientBoostingRegressor(loss='squared_error',
                                     n_estimators=600,
                                     learning_rate=0.05,
                                     min_samples_split=3,
                                     min_samples_leaf=3,
                                     max_depth=3,
                                     max_features=None)

# XGBoost. Best params: alpha=0, colsample_bytree=0.8, eval_metric='rmse', gamma=0.4, learning_rate=0.10000000000000002,
# max_depth=3,min_child_weight=2, n_estimators=250, subsample=0.9
xgboost = XGBRegressor(alpha=0,
                       colsample_bytree=0.8,
                       eval_metric='rmse',
                       gamma=0.4,
                       learning_rate=0.10000000000000002,
                       max_depth=3,
                       min_child_weight=2,
                       n_estimators=250,
                       subsample=0.9)

# LightGBM. Best params: colsample_bytree=0.8, gamma=0.4, learning_rate=0.05, max_bin=1023, max_depth=9, min_child_weight=2,
# min_samples_leaf=3, n_estimators=500, objective='regression', reg_alpha=0.001, subsample=0.8, subsample_freq=2
lightgbm = LGBMRegressor(colsample_bytree=0.8,
                         gamma=0.4,
                         learning_rate=0.05,
                         max_bin=1023,
                         max_depth=9,
                         min_child_weight=2,
                         min_samples_leaf=3,
                         n_estimators=500,
                         objective='regression',
                         reg_alpha=0.001,
                         subsample=0.8,
                         subsample_freq=2)

# RandomForest
random_forest = RandomForestRegressor(n_estimators=1000,
                                      max_depth=13,
                                      max_features=0.6,
                                      min_samples_leaf=1,
                                      min_samples_split=2)

In [51]:
lasso.fit(X_train, y_train)
ridge.fit(X_train, y_train)
elastic.fit(X_train, y_train)
kernelridge.fit(X_train, y_train)
gradient.fit(X_train, y_train)
xgboost.fit(X_train, y_train)
lightgbm.fit(X_train, y_train)
#random_forest.fit(X_train, y_train)

[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] min_data_in_leaf is set with min_child_samples=20, will be overridden by min_samples_leaf=3. Current value: min_data_in_leaf=3
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] min_data_in_leaf is set with min_child_samples=20, will be overridden by min_samples_leaf=3. Current value: min_data_in_leaf=3
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000638 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1043
[LightGBM] [Info] Number of data points in the train set: 1204, number of used features: 10
[LightGBM] [Info] Start training from score 14343563.122924
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


LGBMRegressor(colsample_bytree=0.8, gamma=0.4, learning_rate=0.05, max_bin=1023,
              max_depth=9, min_child_weight=2, min_samples_leaf=3,
              n_estimators=500, objective='regression', reg_alpha=0.001,
              subsample=0.8, subsample_freq=2)

In [ ]:
regression_metrics(lasso.predict(X_train), y_train)
print('=' * 25)
regression_metrics(lasso.predict(X_test), y_test)

In [ ]:
regression_metrics(ridge.predict(X_train), y_train)
print('=' * 25)
regression_metrics(ridge.predict(X_test), y_test)

In [ ]:
regression_metrics(elastic.predict(X_train), y_train)
print('=' * 25)
regression_metrics(elastic.predict(X_test), y_test)

In [ ]:
regression_metrics(kernelridge.predict(X_train), y_train)
print('=' * 25)
regression_metrics(kernelridge.predict(X_test), y_test)

In [ ]:
regression_metrics(gradient.predict(X_train), y_train)
print('=' * 25)
regression_metrics(gradient.predict(X_test), y_test)

In [ ]:
regression_metrics(xgboost.predict(X_train), y_train)
print('=' * 25)
regression_metrics(xgboost.predict(X_test), y_test)

In [ ]:
regression_metrics(lightgbm.predict(X_train), y_train)
print('=' * 25)
regression_metrics(lightgbm.predict(X_test), y_test)

In [52]:
class AveragingModel(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    def fit(self, X, y):
        self.models_ = [clone(model) for model in self.models]
        
        for model in self.models_:
            model.fit(X, y)
        
        return self
    
    def predict(self, X):
        predictions = numpy.column_stack([model.predict(X) for model in self.models_])
        return numpy.mean(predictions, axis=1)

In [53]:
av_model = AveragingModel([kernelridge, gradient, xgboost, lightgbm])
av_model.fit(X_train, y_train)

[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] min_data_in_leaf is set with min_child_samples=20, will be overridden by min_samples_leaf=3. Current value: min_data_in_leaf=3
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] min_data_in_leaf is set with min_child_samples=20, will be overridden by min_samples_leaf=3. Current value: min_data_in_leaf=3
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000232 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1043
[LightGBM] [Info] Number of data points in the train set: 1204, number of used features: 10
[LightGBM] [Info] Start training from score 14343563.122924
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


AveragingModel(models=[KernelRidge(coef0=31, gamma=0.001, kernel='polynomial'),
                       GradientBoostingRegressor(learning_rate=0.05,
                                                 min_samples_leaf=3,
                                                 min_samples_split=3,
                                                 n_estimators=600),
                       XGBRegressor(alpha=0, base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None, colsample_bytree=0.8,
                                    device=None, early_stopping_rounds=None,
                                    e...
                                    min_child_weight=2, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=250,
                                    n_jobs=None, num_parallel_tree=None, ...),
                       LGBMRegressor(colsample_bytree=0.8, gamma=0.4,
                                     learning_rate=0.05, max_bin=1023,
                                     max_depth=9, min_child_weight=2,
                                     min_samples_leaf=3, n_estimators=500,
                                     objective='regression', reg_alpha=0.001,
                                     subsample=0.8, subsample_freq=2)])

In [56]:
regression_metrics(av_model.predict(X_train), y_train)
print('=' * 25)
regression_metrics(av_model.predict(X_test), y_test)

[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] min_data_in_leaf is set with min_child_samples=20, will be overridden by min_samples_leaf=3. Current value: min_data_in_leaf=3
RMSE: 3736211.3061936176
Normalized RMSE: 0.0415365348103793
log RMSE: 0.4722946028632217
MAPE: 0.38423777946404475
R2 score: 0.9395166831794177
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] min_data_in_leaf is set with min_child_samples=20, will be overridden by min_samples_leaf=3. Current value: min_data_in_leaf=3
RMSE: 6689582.352995455
Normalized RMSE: 0.0894927405083004
log RMSE: 0.6177580957238272
MAPE: 0.5698716517993325
R2 score: 0.7695213874486447


In [57]:
pred = show_predictions(av_model)
pred[abs(pred['Test value'] - pred['Predicted value']) > 5000000]

[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] min_data_in_leaf is set with min_child_samples=20, will be overridden by min_samples_leaf=3. Current value: min_data_in_leaf=3


,Name,Test value,Predicted value
0,Paulo Dybala,20000000,27136633.509738
1,Pervis Estupiñán,30000000,24621461.86845
2,Aimar Oroz,15000000,7063245.031612
3,Maximilian Mittelstädt,17000000,25665437.145351
8,João Félix,30000000,47477395.497802
...,...,...,...
281,Harry Maguire,18000000,28861748.385563
285,Robin Hack,8000000,25856235.785819
288,Nathan Tella,23000000,17122851.174799
291,Dominic Calvert-Lewin,22000000,30732803.638824


In [58]:
with open('page/final_model.pkl', 'wb') as file:
    dill.dump(av_model, file)

In [ ]:
dataframe = pd.DataFrame(columns=feature_names)

In [ ]:
dataframe.to_csv(r'./page/online_data.csv', index=False)